Перед запуском убедитесь, что в корне проекта есть файл .env и в нем заполнены выданные вам креды подключения к базам данных и хранилищу

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv
from sqlalchemy import create_engine

In [3]:
# подгружаем .env
load_dotenv()

True

In [4]:
# Считываем все креды
src_host = os.environ.get('DB_SOURCE_HOST')
src_port = os.environ.get('DB_SOURCE_PORT')
src_username = os.environ.get('DB_SOURCE_USER')
src_password = os.environ.get('DB_SOURCE_PASSWORD')
src_db = os.environ.get('DB_SOURCE_NAME') 

dst_host = os.environ.get('DB_DESTINATION_HOST')
dst_port = os.environ.get('DB_DESTINATION_PORT')
dst_username = os.environ.get('DB_DESTINATION_USER')
dst_password = os.environ.get('DB_DESTINATION_PASSWORD')
dst_db = os.environ.get('DB_DESTINATION_NAME')

s3_bucket = os.environ.get('S3_BUCKET_NAME')
s3_access_key = os.environ.get('AWS_ACCESS_KEY_ID')
s3_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

In [5]:
# Создадим соединения
src_conn = create_engine(f'postgresql://{src_username}:{src_password}@{src_host}:{src_port}/{src_db}')
dst_conn = create_engine(f'postgresql://{dst_username}:{dst_password}@{dst_host}:{dst_port}/{dst_db}')

In [7]:
# from sqlalchemy import MetaData, Table, Column, String, Float, Integer, DateTime, UniqueConstraint, inspect
# metadata = MetaData()
# users_churn_table = Table(
#     'users_churn',
#     metadata,
#     Column('id', Integer, primary_key=True, autoincrement=True),
#     Column('customer_id', String),
#     Column('begin_date', DateTime),
#     Column('end_date', DateTime),
#     Column('type', String),
#     Column('paperless_billing', String),
#     Column('payment_method', String),
#     Column('monthly_charges', Float),
#     Column('total_charges', Float),
#     Column('internet_service', String),
#     Column('online_security', String),
#     Column('online_backup', String),
#     Column('device_protection', String),
#     Column('tech_support', String),
#     Column('streaming_tv', String),
#     Column('streaming_movies', String),
#     Column('gender', String),
#     Column('senior_citizen', Integer),
#     Column('partner', String),
#     Column('dependents', String),
#     Column('multiple_lines', String),
#     Column('target', Integer),
#     UniqueConstraint('customer_id', name='unique_customer_constraint')
#     )

# if not inspect(dst_conn).has_table(users_churn_table.name): 
#     metadata.create_all(dst_conn)

In [8]:
from sqlalchemy import inspect
inspector = inspect(dst_conn)

for table_name in inspector.get_table_names():
   print(table_name)

users_churn


In [12]:
# Пример выгрузки данных из БД
SQL = f'select * from users_churn'
data = pd.read_sql(SQL, dst_conn)
data

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
0,17,8191-XWSZG,2015-10-01,NaT,One year,No,Mailed check,20.65,1022.95,None,...,None,None,None,None,Female,0,No,No,No,0
1,59,3957-SQXML,2017-04-01,NaT,Two year,No,Credit card (automatic),24.95,894.30,None,...,None,None,None,None,Female,0,Yes,Yes,Yes,0
2,147,6837-BJYDQ,2019-11-01,NaT,One year,No,Mailed check,19.60,61.35,None,...,None,None,None,None,Male,0,No,No,No,0
3,481,0486-LGCCH,2019-03-01,NaT,Two year,No,Mailed check,19.65,225.75,None,...,None,None,None,None,Male,0,Yes,Yes,No,0
4,1001,8357-EQXFO,2019-04-01,2019-11-01,Month-to-month,Yes,Electronic check,95.35,660.90,Fiber optic,...,Yes,No,Yes,Yes,Female,0,No,No,No,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,7039,6840-RESVB,2018-02-01,NaT,One year,Yes,Mailed check,84.80,1990.50,DSL,...,Yes,Yes,Yes,Yes,Male,0,Yes,Yes,Yes,0
7039,7040,2234-XADUH,2014-02-01,NaT,One year,Yes,Credit card (automatic),103.20,7362.90,Fiber optic,...,Yes,No,Yes,Yes,Female,0,Yes,Yes,Yes,0
7040,7041,4801-JZAZL,2019-03-01,NaT,Month-to-month,Yes,Electronic check,29.60,346.45,DSL,...,No,No,No,No,Female,0,Yes,Yes,None,0
7041,7042,8361-LTMKD,2019-07-01,2019-11-01,Month-to-month,Yes,Mailed check,74.40,306.60,Fiber optic,...,No,No,No,No,Male,1,Yes,No,Yes,1


In [10]:
data.shape

(7043, 22)

In [27]:
src_conn.dispose()
dst_conn.dispose()

А дальше, творите!